## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/rwhited/Desktop/Classwork/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,-26.45,86,100,9.26,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,14.07,85,59,3.65,broken clouds
2,2,Hithadhoo,MV,-0.6000,73.0833,81.36,73,56,2.73,light rain
3,3,Mataura,NZ,-46.1927,168.8643,60.01,64,100,8.99,overcast clouds
4,4,Provideniya,RU,64.3833,-173.3000,10.20,93,94,5.46,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hithadhoo,MV,-0.6000,73.0833,81.36,73,56,2.73,light rain
13,13,Luderitz,NaN,-26.6481,15.1594,75.38,56,0,17.36,clear sky
14,14,Tautira,PF,-17.7333,-149.1500,78.01,74,36,23.00,moderate rain
17,17,Cape Town,ZA,-33.9258,18.4232,82.00,65,20,17.27,few clouds
18,18,Arraial Do Cabo,BR,-22.9661,-42.0278,87.80,70,20,18.41,few clouds
22,22,Bama,NG,11.5221,13.6856,90.00,12,32,10.69,scattered clouds
27,27,Atuona,PF,-9.8000,-139.0333,77.76,79,0,16.40,clear sky
29,29,Rikitea,PF,-23.1203,-134.9692,77.83,75,88,15.52,overcast clouds
40,40,East London,ZA,-33.0153,27.9116,82.40,74,20,16.11,few clouds
41,41,Gurupa,BR,-1.4050,-51.6400,80.73,83,77,3.36,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                201
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,81.36,light rain,-0.6000,73.0833,
14,Tautira,PF,78.01,moderate rain,-17.7333,-149.1500,
17,Cape Town,ZA,82.00,few clouds,-33.9258,18.4232,
18,Arraial Do Cabo,BR,87.80,few clouds,-22.9661,-42.0278,
22,Bama,NG,90.00,scattered clouds,11.5221,13.6856,
27,Atuona,PF,77.76,clear sky,-9.8000,-139.0333,
29,Rikitea,PF,77.83,overcast clouds,-23.1203,-134.9692,
40,East London,ZA,82.40,few clouds,-33.0153,27.9116,
41,Gurupa,BR,80.73,broken clouds,-1.4050,-51.6400,
42,Lakatoro,VU,81.66,light rain,-16.0999,167.4164,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))